In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

snap = pd.read_csv('./ReducedData/snapframe.csv')
extra_punts = pd.read_csv('./ReducedData/BounceCatch.csv').merge(snap,how='right',indicator=True).query('_merge=="right_only"').drop(columns=['_merge'])[['gameId','playId','snap_frame']]
extra_punts.rename({'snap_frame':'frameId'},axis=1,inplace=True)

play_data = pd.read_csv('./data/plays.csv')
discard_results = ['Non-Special Teams Result']#,'Blocked Punt']
punt_plays = play_data[(play_data['specialTeamsPlayType']=='Punt')&(~play_data['specialTeamsResult'].isin(discard_results))]

In [2]:
# 2018 Football Tracking Data
tracking_game = pd.read_csv('./data/tracking2018.csv')
punt_fbtrack_18 = tracking_game.merge(extra_punts)
punt_fbtrack_18['time'] = pd.to_datetime(punt_fbtrack_18['time'])

# 2019 Football Tracking Data
tracking_game = pd.read_csv('./data/tracking2019.csv')
punt_fbtrack_19 = tracking_game.merge(extra_punts)
punt_fbtrack_19['time'] = pd.to_datetime(punt_fbtrack_19['time'])

# 2020 Football Tracking Data
tracking_game = pd.read_csv('./data/tracking2020.csv')
punt_fbtrack_20 = tracking_game.merge(extra_punts)
punt_fbtrack_20['time'] = pd.to_datetime(punt_fbtrack_20['time'])

# Combine all above tracking data
punt_fbtrack = pd.concat([punt_fbtrack_18,punt_fbtrack_19,punt_fbtrack_20]) \
                 .sort_values(by=['gameId','playId','frameId']) \
                 .drop(columns=['o','dir','displayName','jerseyNumber']) \
                 .reset_index(drop=True)

In [3]:
# We want to preserve the handedness of plays and players when flipping the tracking data
# We want to transform plays where playDirection == 'left'
# Reflect the x and y position, and rotate orientation and direction.

def flip_play_direction(df):
    df_flipped = df.copy()
    df_flipped['to_flip'] = (df['playDirection']=='left')
    
    df_flipped['x'] = df_flipped['x']*(1-df_flipped['to_flip']) + (120-df_flipped['x'])*(df_flipped['to_flip'])
    df_flipped['y'] = df_flipped['y']*(1-df_flipped['to_flip']) + (160/3-df_flipped['y'])*(df_flipped['to_flip'])
    
    # When orientation of players is included, flipping field is equal to a 180 degree rotation.
    # Mod by 360 degrees to keep values in the range [0,360)
    orientation_cols = [col for col in df_flipped.columns if col in ['o','dir']]
    for col in orientation_cols:
        df_flipped[col] = (180*(df_flipped['to_flip']) + df_flipped[col])%360

    return df_flipped.drop(columns=['to_flip'])

flipped_track = flip_play_direction(punt_fbtrack)
fb_loc = flipped_track[flipped_track['team']=='football']
players = flipped_track[flipped_track['team'].isin(['home','away'])]

In [4]:
punter = players[players['position'].isin(['P','K'])].sort_values('x',ascending=False).groupby(['gameId','playId']).last().reset_index()
temp = players.merge(punter[['gameId','playId','team']],on=['gameId','playId'],suffixes=('','_punt'))
punt_team = temp[temp['team']==temp['team_punt']].copy()
rec_team = temp[temp['team']!=temp['team_punt']].copy()

In [5]:
no_punter = punt_team.merge(punter,how='left',indicator=True).query('_merge=="left_only"').drop(columns = ['_merge'])[['gameId','playId','x','y']].copy()
no_punter['num'] = no_punter.groupby(['gameId','playId']).cumcount()
punt_team_pos = pd.pivot_table(no_punter,values=['x','y'],index=['gameId','playId'],columns='num')
punt_team_pos.columns  = [f'x_p{i}' for i in range(10)] + [f'y_p{i}' for i in range(10)]
punt_team_pos = punt_team_pos[[punt_team_pos.columns[i] for i in np.array([[j, j+10] for j in range(10)]).reshape((20,))]].reset_index().round(2)
punt_team_pos

,gameId,playId,x_p0,y_p0,x_p1,y_p1,x_p2,y_p2,x_p3,y_p3,...,x_p5,y_p5,x_p6,y_p6,x_p7,y_p7,x_p8,y_p8,x_p9,y_p9
0,2018090901,118,41.77,9.04,41.50,31.12,38.10,28.92,38.53,32.21,...,39.27,27.36,41.45,28.98,42.34,30.08,42.09,44.88,41.36,31.68
1,2018090901,4965,32.85,22.56,35.28,24.17,33.67,23.12,34.33,45.65,...,32.19,21.35,32.56,25.42,29.34,25.91,33.29,26.36,32.32,26.95
2,2018090907,2357,24.54,22.56,20.92,31.00,23.90,27.08,25.07,29.31,...,21.58,26.21,20.87,32.61,24.22,30.41,24.16,28.60,24.59,46.29
3,2018090907,2877,60.90,20.61,61.76,22.03,64.74,21.70,65.59,23.43,...,61.39,26.00,64.77,24.48,65.20,22.47,65.09,44.72,65.30,7.39
4,2018091000,3527,25.46,30.92,24.11,32.45,25.31,31.78,23.63,27.28,...,25.67,28.92,25.59,33.97,25.70,24.36,21.64,28.41,26.28,29.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2020122713,474,77.19,23.97,76.21,21.95,76.25,24.89,74.74,26.49,...,72.90,22.51,76.03,25.85,74.46,21.53,77.02,10.56,76.93,40.02
356,2021010300,2755,63.49,28.41,60.88,27.25,63.24,30.23,64.47,29.33,...,63.24,30.96,60.74,30.56,63.24,27.47,60.99,31.54,63.17,46.80
357,2021010304,2404,48.93,26.79,44.22,28.42,45.53,29.12,48.08,25.04,...,48.73,48.15,47.97,28.01,48.00,28.78,48.14,25.72,49.00,7.21
358,2021010309,2235,41.87,20.29,44.43,23.54,44.15,6.58,44.45,45.05,...,43.73,22.44,40.36,25.07,44.00,26.48,44.00,24.99,43.60,21.11


In [6]:
test = rec_team[['gameId','playId','x','y']].copy()
test['num'] = test.groupby(['gameId','playId']).cumcount()
rec_pos = pd.pivot_table(test,index=['gameId','playId'],columns='num',values=['x','y'])
rec_pos.columns = [f'x_r{i}' for i in range(11)] + [f'y_r{i}' for i in range(11)]
rec_pos = rec_pos[[rec_pos.columns[i] for i in np.array([[j, j+11] for j in range(11)]).reshape((22,))]].reset_index().round(2)
rec_pos

,gameId,playId,x_r0,y_r0,x_r1,y_r1,x_r2,y_r2,x_r3,y_r3,...,x_r6,y_r6,x_r7,y_r7,x_r8,y_r8,x_r9,y_r9,x_r10,y_r10
0,2018090901,118,43.47,30.81,43.41,28.89,47.02,28.68,43.56,9.95,...,43.83,45.17,43.24,27.90,43.77,26.28,87.15,30.43,43.43,32.64
1,2018090901,4965,35.92,8.28,36.42,22.88,36.36,28.19,36.21,20.07,...,36.58,25.75,38.92,25.67,75.77,24.12,36.25,26.80,36.90,5.86
2,2018090907,2357,25.93,26.30,26.82,45.20,28.97,24.30,26.41,34.61,...,31.56,19.37,27.97,22.98,26.77,46.70,26.64,32.17,28.82,30.86
3,2018090907,2877,68.49,44.04,66.99,18.58,66.85,20.88,70.44,19.78,...,67.59,7.54,70.14,24.14,66.94,25.47,66.97,28.09,66.60,22.36
4,2018091000,3527,27.31,27.23,27.21,23.45,26.69,25.74,27.21,35.70,...,27.38,24.53,27.10,34.40,27.21,30.79,82.59,24.70,26.99,33.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2020122713,474,102.41,23.92,78.86,17.48,78.65,28.20,86.05,39.53,...,78.88,25.78,81.77,30.29,78.91,22.65,82.52,20.43,82.63,23.91
356,2021010300,2755,65.79,32.48,65.03,30.07,65.24,25.43,67.98,30.32,...,65.29,31.13,65.00,27.23,65.59,10.44,67.91,28.15,100.64,27.73
357,2021010304,2404,50.26,30.75,49.99,25.68,50.62,46.24,49.67,23.11,...,50.55,24.87,50.21,27.73,51.44,7.66,58.12,27.39,50.56,48.77
358,2021010309,2235,45.90,29.68,48.68,24.89,51.26,7.25,45.77,24.93,...,45.83,17.79,52.07,44.27,88.32,26.00,46.62,27.51,45.79,20.68


In [7]:
fb_loc_update = fb_loc[['gameId','playId','x','y','playDirection']].merge(punt_plays[['gameId','playId','absoluteYardlineNumber']])
fb_loc_update['x_yl'] = fb_loc_update['absoluteYardlineNumber']
fb_loc_update.loc[fb_loc_update['playDirection']=='left','x_yl'] = 120 - fb_loc_update.loc[fb_loc_update['playDirection']=='left','absoluteYardlineNumber']
fb_loc_fin = fb_loc_update[['gameId','playId','x_yl','y']].rename({'x_yl':'x_fb','y':'y_fb'},axis=1)

In [8]:
all_pos = punter[['gameId','playId','nflId','x','y']].rename({'nflId':'nflId_punt','x':'x_punt','y':'y_punt'},axis=1) \
    .merge(punt_team_pos).merge(rec_pos).merge(fb_loc_fin).round(2)
all_pos

,gameId,playId,nflId_punt,x_punt,y_punt,x_p0,y_p0,x_p1,y_p1,x_p2,...,x_r7,y_r7,x_r8,y_r8,x_r9,y_r9,x_r10,y_r10,x_fb,y_fb
0,2018090901,118,42333.0,27.87,29.71,41.77,9.04,41.50,31.12,38.10,...,43.24,27.90,43.77,26.28,87.15,30.43,43.43,32.64,43,30.08
1,2018090901,4965,34723.0,21.80,24.44,32.85,22.56,35.28,24.17,33.67,...,38.92,25.67,75.77,24.12,36.25,26.80,36.90,5.86,36,24.16
2,2018090907,2357,46316.0,12.00,29.11,24.54,22.56,20.92,31.00,23.90,...,27.97,22.98,26.77,46.70,26.64,32.17,28.82,30.86,26,29.40
3,2018090907,2877,46316.0,52.55,23.38,60.90,20.61,61.76,22.03,64.74,...,70.14,24.14,66.94,25.47,66.97,28.09,66.60,22.36,66,23.67
4,2018091000,3527,43524.0,12.43,31.00,25.46,30.92,24.11,32.45,25.31,...,27.10,34.40,27.21,30.79,82.59,24.70,26.99,33.35,27,29.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2020122713,474,33338.0,63.75,23.54,77.19,23.97,76.21,21.95,76.25,...,81.77,30.29,78.91,22.65,82.52,20.43,82.63,23.91,78,23.77
356,2021010300,2755,46903.0,50.01,30.43,63.49,28.41,60.88,27.25,63.24,...,65.00,27.23,65.59,10.44,67.91,28.15,100.64,27.73,65,29.34
357,2021010304,2404,48139.0,36.20,24.96,48.93,26.79,44.22,28.42,45.53,...,50.21,27.73,51.44,7.66,58.12,27.39,50.56,48.77,50,26.70
358,2021010309,2235,45603.0,30.65,24.38,41.87,20.29,44.43,23.54,44.15,...,52.07,44.27,88.32,26.00,46.62,27.51,45.79,20.68,45,23.57


In [82]:
all_pos.to_csv('./ReducedData/ExtraPuntLocations.csv',index=False)